In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("CSV Files/8_train.csv")
test_df = pd.read_csv("CSV Files/8_test.csv")

In [3]:
X_train = train_df.drop("label", axis=1).values
y_train = train_df["label"].values

X_test = test_df.values

In [4]:
print(X_train.shape, y_train.shape, X_test.shape)

(42000, 784) (42000,) (28000, 784)


In [5]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [6]:
class MNISTTrainDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = self.X[idx].reshape(28,28) / 255.0
        img = img.unsqueeze(0)   # [1,28,28]
        label = self.y[idx]
        return img, label

class MNISTTestDataset(Dataset):
    def __init__(self, X):
        self.X = X
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = self.X[idx].reshape(28,28) / 255.0
        img = img.unsqueeze(0)
        return img

In [7]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train_tensor, y_train_tensor, test_size=0.1, random_state=42)

train_ds = MNISTTrainDataset(X_tr, y_tr)
val_ds   = MNISTTrainDataset(X_val, y_val)
test_ds  = MNISTTestDataset(X_test_tensor)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)

In [8]:
class DigitClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [10]:
model = DigitClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for imgs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss={avg_loss:.4f}")

Epoch [1/5], Loss=0.3479
Epoch [2/5], Loss=0.1363
Epoch [3/5], Loss=0.0902
Epoch [4/5], Loss=0.0657
Epoch [5/5], Loss=0.0481


In [12]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

print(f"Validation Accuracy: {100*correct/total:.2f}%") 

Validation Accuracy: 96.88%


In [13]:
model.eval()
predictions = []
with torch.no_grad():
    for imgs in test_loader:
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())


In [14]:
submission = pd.DataFrame({
    "ImageId": range(1, len(predictions)+1),
    "Label": predictions
})
submission.to_csv("submission.csv", index=False)